In [1]:
# Import the relevant libraries
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds


In [2]:
# Data
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True,as_supervised=True)

In [3]:
mnist_train, mnist_test = mnist_dataset['train'],mnist_dataset['test']

num_validation_samples = 0.1* mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64) # cast() change the vale to the given datatype

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image , label):
    image = tf.cast(image, tf.float32)
    image /= 255. # . represents we want the float value
    return image,label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffle_trained_and_validaion_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffle_trained_and_validaion_data.take(num_validation_samples)
train_data = shuffle_trained_and_validaion_data.skip(num_validation_samples)


BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples) # Single batch, having size equal to number of validation samples

test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data)) # iter() creates an object which can be itreted one element at a time


In [4]:
# Outline the model
input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)), # Flatten() transform a tensor into the a vector
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(output_size,activation='softmax')
                            ])

In [5]:
# Choose the optimizer and loss function

model.compile(optimizer ='adam',loss ='sparse_categorical_crossentropy', metrics=['accuracy'])

In [6]:

NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs,validation_targets), validation_steps=10,verbose=2 )

Epoch 1/5
540/540 - 12s - loss: 0.4175 - accuracy: 0.8824 - val_loss: 0.2385 - val_accuracy: 0.9357
Epoch 2/5
540/540 - 16s - loss: 0.1798 - accuracy: 0.9481 - val_loss: 0.1727 - val_accuracy: 0.9518
Epoch 3/5
540/540 - 16s - loss: 0.1368 - accuracy: 0.9599 - val_loss: 0.1447 - val_accuracy: 0.9592
Epoch 4/5
540/540 - 15s - loss: 0.1091 - accuracy: 0.9678 - val_loss: 0.1258 - val_accuracy: 0.9657
Epoch 5/5
540/540 - 16s - loss: 0.0926 - accuracy: 0.9720 - val_loss: 0.1100 - val_accuracy: 0.9675


In [7]:
# test the model
test_loss , test_accuracy = model.evaluate(test_data)

      1/Unknown - 2s 2s/step - loss: 0.1109 - accuracy: 0.9678

In [8]:
print('Test loss: {0:.2f} , Test accuracy: {1:.2f}'.format(test_loss , test_accuracy*100.))

Test loss: 0.11 , Test accuracy: 96.78
